In [6]:
repo_directory = '/Users/iaincarmichael/Dropbox/Research/law/law-net/'

data_dir = '/Users/iaincarmichael/Documents/courtlistener/data/'

import numpy as np
import sys

# graph package
import igraph as ig

# our code
sys.path.append(repo_directory + 'code/')
from setup_data_dir import setup_data_dir, make_subnetwork_directory
from pipeline.download_data import download_bulk_resource, download_master_edgelist, download_scdb
from helpful_functions import case_info

sys.path.append(repo_directory + 'explore/vertex_metrics_experiment/code/')
from make_network_data import *
from make_graph import make_graph
from bag_of_words import make_tf_idf
from make_snapshots import make_snapshot_vertex_metrics

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'


# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# set up the data directory

In [ ]:
setup_data_dir(data_dir)

In [ ]:
make_subnetwork_directory(data_dir, network_name)

# data download

## get opinion and cluster files from CourtListener

opinions/cluster files are saved in data_dir/raw/court/ 

In [ ]:
download_op_and_cl_files(data_dir, network_name)

## get the master edgelist from CL

master edgelist is saved in data_dir/raw/

In [ ]:
download_master_edgelist(data_dir)

## download scdb data from SCDB

scdb data is saved in data_dir/scdb

In [ ]:
download_scdb(data_dir)

# network data

## make the case metadata and edgelist

- add the raw case metadata data frame to the raw/ folder
- remove cases missing scdb ids
- remove detroit lumber case
- get edgelist of cases within desired subnetwork
- save case metadata and edgelist to the experiment_dir/

In [ ]:
# create the raw case metadata data frame in the raw/ folder
make_subnetwork_raw_case_metadata(data_dir, network_name)

In [ ]:
# create clean case metadata and edgelist from raw data
clean_metadata_and_edgelist(data_dir, network_name)

## make graph

creates the network with the desired case metadata and saves it as a .graphml file in experiment_dir/ 

In [ ]:
make_graph(subnet_dir, network_name)

# NLP data

## make case text files

grabs the opinion text for each case in the network and saves them as a text file in experiment_dir/textfiles/

In [ ]:
# make the textfiles for give court
make_network_textfiles(data_dir, network_name)

## make tf-idf matrix

creates the tf-idf matrix for the corpus of cases in the network and saves them to  subnet_dir + 'nlp/'

In [3]:
%time make_tf_idf(text_dir, subnet_dir + 'nlp/', min_df=0, max_df=1)

CPU times: user 3min 45s, sys: 18.3 s, total: 4min 3s
Wall time: 4min 20s


# Load network

In [8]:
# load the graph
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

In [9]:
G.summary()

'IGRAPH DN-- 27885 234312 -- \n+ attr: court (v), id (v), issueArea (v), name (v), year (v)'

# compute snapshots

In [10]:
vertex_metrics = ['indegree', 'outdegree', 'degree',
                   'd_pagerank', 'authorities', 'hubs']

# add recent citations
vertex_metrics += ['recentcite_' + str(t) for t in 5 * np.arange(1, 6+1)]

active_years = range(1900, 2015 + 1)

In [11]:
make_snapshot_vertex_metrics(G, active_years, vertex_metrics, subnet_dir)

year 1900, (2/117) at 22:22:19
year 1902, (4/117) at 22:22:21
year 1906, (8/117) at 22:22:23
year 1914, (16/117) at 22:22:29
year 1930, (32/117) at 22:22:46
year 1962, (64/117) at 22:23:36
CPU times: user 2min 52s, sys: 5.9 s, total: 2min 58s
Wall time: 3min 7s
